In [1]:
# Jessica Gallo
# CSC767 Neural Networks & Deep Learning
# Project 2
# Part E

In [2]:
# This is the original code with the original optimizers (Adam & SGD)
# This was run with the original dataset under limited data files:
# 0.txt, 6.txt and 8.txt.

In [3]:
# Originally cloned the repository onto local machine via:
# Cloning github repository
# %%bash
# git clone https://github.com/""""""/Lightweighted-CNN-for-Document-Classification

In [4]:
#! /usr/bin/env python
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from helper import data_helpers
from text_cnn.LW_text_cnn import TextCNN
from tensorflow.contrib import learn
import yaml
import math
import sys

C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [5]:
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

In [6]:
# Parameters
# ==================================================

In [7]:
# Data loading params
tf.compat.v1.flags.DEFINE_float("dev_sample_percentage", 0.2, "Percentage of the training data to use for validation")

In [8]:
# Model Hyperparameters
tf.compat.v1.flags.DEFINE_boolean("enable_word_embeddings", False, "Enable/disable the word embedding (default: True)")
tf.compat.v1.flags.DEFINE_integer("embedding_dim", 200, "Dimensionality of character embedding (default: 128)")
tf.compat.v1.flags.DEFINE_string("filter_sizes", "2,3,5", "Comma-separated filter sizes (default: '2,3,5')")
tf.compat.v1.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 120)")
tf.compat.v1.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.compat.v1.flags.DEFINE_float("l2_reg_lambda", 0.0001, "L2 regularization lambda (default: 0.0)")

In [9]:
# Training parameters
tf.compat.v1.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 64)")
tf.compat.v1.flags.DEFINE_integer("num_epochs",10, "Number of training epochs (default: 10)")
tf.compat.v1.flags.DEFINE_integer("evaluate_every", 50, "Evaluate model on dev set after this many steps (default: 100)")
tf.compat.v1.flags.DEFINE_integer("checkpoint_every", 50, "Save model after this many steps (default: 100)")
tf.compat.v1.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

In [10]:
# Misc Parameters
tf.compat.v1.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.compat.v1.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.compat.v1.flags.DEFINE_float("decay_coefficient", 2.5, "Decay coefficient (default: 2.5)")

In [11]:
FLAGS = tf.compat.v1.flags.FLAGS

In [12]:
YML_PATH = os.path.join(ROOT_DIR, "train/helper/config.yml")
print("***********************YML_PATH", YML_PATH)

***********************YML_PATH C:\Users\User\Lightweighted-CNN-for-Document-Classification\train/helper/config.yml


In [13]:
with open(YML_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

In [14]:
# ===================================================================================
# added this because of 'Unrecognizable flag error: Unknown command line flag f'
tf.compat.v1.flags.DEFINE_string('f','','')
# ===================================================================================

dataset_name = cfg["datasets"]["default"]
if FLAGS.enable_word_embeddings and cfg['word_embeddings']['default'] is not None:
    embedding_name = cfg['word_embeddings']['default']
    embedding_dimension = cfg['word_embeddings'][embedding_name]['dimension']
else:
    embedding_dimension = FLAGS.embedding_dim

In [15]:
# Data Preparation
# ==================================================

In [16]:
# Load data
print("Loading data...")
datasets = None
if dataset_name == "tobacco":
    data_path = os.path.join(ROOT_DIR, cfg["datasets"][dataset_name]['parent_dir'] +'/')
    print("*************data_path", data_path)    
    datasets = data_helpers.get_datasets_tobacco(data_path)

elif dataset_name == "localdata":
    datasets = data_helpers.get_datasets_localdata(container_path=cfg["datasets"][dataset_name]["container_path"],
                                                     categories=cfg["datasets"][dataset_name]["categories"],
                                                     shuffle=cfg["datasets"][dataset_name]["shuffle"],
                                                     random_state=cfg["datasets"][dataset_name]["random_state"])

Loading data...
*************data_path C:\Users\User\Lightweighted-CNN-for-Document-Classification\data/tobacco-data/
['0.txt', '6.txt', '8.txt']
The Data before flattening:  [['SURGEON GENERAL’S WARNING: Smoking', 'By Pregnant Women May Result in Fetal', 'injury, Premature Birth, And Low Birth Weight.', 'A r 7', '<a:u:', 'Jimmy’s Grille', '32!] Snullr Birch 322-5334', 'BARS “HER“ LIVE MUS_IC', "Gel it uin' will] the lunk~ azz sounds n1 « MW ' -", 'the ﬁne and only Dnlsaliu. n’u keep ‘ SNAKE PIT SynergY-ﬂhe “W New “3 b9- CRICKET on 11-:', '. your heoly shakin’ until lhe wee hours. ace E. l th Ave. 8 l-la r. Fridays, lfs B_alllsuc with high no. Easy , m', 'not“ misslhis show on Tuesday, Snake Pit Fridays introduces the energy dance with DIs Amtrax. Sizzlin’ Sepgem', 'Saplemher 30. Brnughiin you by Camel. home grown mlent of Miss Saturdays, Shake! Club, dance, ai‘dweﬁlml', 'Chocolate Thunderpussy. house with Anthrayngribgﬂ . ‘', 'Feast your eyes on chis spectacw Rlc', '[ar show and keep 

In [17]:
x_text, y = data_helpers.load_data_labels(datasets)

In [18]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [19]:
# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

In [20]:
# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 16099
Train/Dev split: 9549/2387


In [21]:
# Training
# ==================================================

In [22]:
# ===========================================================================
# Added to fix error
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# ===========================================================================

with tf.Graph().as_default():
    session_conf = tf.compat.v1.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)

    sess = tf.compat.v1.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=embedding_dimension,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
       
        global_step = tf.compat.v1.Variable(0, name="global_step", trainable=False)

        early_train_op= tf.compat.v1.train.AdamOptimizer(cnn.learning_rate)
        late_train_op = tf.compat.v1.train.MomentumOptimizer(cnn.learning_rate,0.9,use_nesterov=True) 
        
        grads_and_vars_1 = early_train_op.compute_gradients(cnn.loss)
        train_op_1 = early_train_op.apply_gradients(grads_and_vars_1, global_step=global_step)
        
        grads_and_vars_2 = late_train_op.compute_gradients(cnn.loss)
        train_op_2 = late_train_op.apply_gradients(grads_and_vars_2, global_step=global_step)  

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.compat.v1.summary.scalar("loss", cnn.loss)
        acc_summary = tf.compat.v1.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.compat.v1.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.compat.v1.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.compat.v1.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.compat.v1.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.compat.v1.train.Saver(tf.compat.v1.trainable_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.compat.v1.global_variables_initializer())
        if FLAGS.enable_word_embeddings and cfg['word_embeddings']['default'] is not None:
            vocabulary = vocab_processor.vocabulary_
            initW = None
            if embedding_name == 'word2vec':
                # load embedding vectors from the word2vec
                print("Load word2vec file {}".format(cfg['word_embeddings']['word2vec']['path']))
                initW = data_helpers.load_embedding_vectors_word2vec(vocabulary,
                                                                     cfg['word_embeddings']['word2vec']['path'],
                                                                     cfg['word_embeddings']['word2vec']['binary'])
                print("word2vec file has been loaded")
            elif embedding_name == 'glove':
                # load embedding vectors from the glove
                print("Load glove file {}".format(cfg['word_embeddings']['glove']['path']))
                initW = data_helpers.load_embedding_vectors_glove(vocabulary,
                                                                  cfg['word_embeddings']['glove']['path'],
                                                                  embedding_dimension)
                print("glove file has been loaded\n")
            sess.run(cnn.W.assign(initW))

        def train_step(x_batch, y_batch, learning_rate):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob,                
              cnn.learning_rate: learning_rate              
            }
            
            #Conditional optimizer switch
            train_op = train_op_1 if tf.compat.v1.train.global_step(sess, global_step) <=(train_input_size/FLAGS.batch_size)*3 else train_op_2
                        
            _, step, summaries, loss, accuracy = sess.run(                
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],                
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            
            print("{}: step {}, loss {:g}, acc {:g}, learning_rate {:g}"
                  .format(time_str, step, loss, accuracy, learning_rate))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
                

        train_input_size=len(list(zip(x_train, y_train)))
        print('Trainning input set: x_train, y_train',train_input_size)
        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        
        # It uses dynamic learning rate with a high value at the beginning to speed up the training
        max_learning_rate = 0.004
        min_learning_rate = 0.0001        
        decay_speed = FLAGS.decay_coefficient*len(y_train)/FLAGS.batch_size
        
        # Training loop. For each batch...
        counter = 0
        print("*********Trainable PARAMETERS***********",np.sum([np.prod(v.get_shape().as_list()) for v in 
                                                                 tf.compat.v1.trainable_variables()]))
        for batch in batches:
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-counter/decay_speed)
            counter += 1
            x_batch, y_batch = zip(*batch)      
            
            if tf.compat.v1.train.global_step(sess, global_step) == ((train_input_size/FLAGS.batch_size)*3)+1: ##(2534*3)+1:
                min_learning_rate = 0.0035
                
            train_step(x_batch, y_batch, learning_rate)
            
            current_step = tf.compat.v1.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
(?, 41, 1, 1)
(?, 40, 1, 1)
(?, 42, 200, 1)
(?, 40, 1, 1)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Writing to C:\Users\User\Lightweighted-CNN-for-Document-Classification\train\runs\1607457019

Trainning input set: x_train, y_train 9549
*********Trainable PARAMETERS*********** 3224697
2020-12-08T14:50:25.132350: step 1, loss 1.11376, acc 0.375, learning_rate 0.004
2020-12-08T14:50:26.350850: step 2, loss 1.14688, acc 0.34375, learning_rate 0.00399478
2020-12-08T14:50:27.366876: step 3, loss 1.03256, acc 0.59375, learning_rate 0.00398956
2020-12-08T14:50:28.416820: step 4, loss 1.14423, acc 0.40625, learning_rate 0.00398435
2020-12-08T14:50:29.543361: step 5, loss 1.14244, acc 0.375, learning_r

2020-12-08T14:51:22.824379: step 80, loss 0.863873, acc 0.5625, learning_rate 0.00360812
2020-12-08T14:51:23.440878: step 81, loss 0.781226, acc 0.59375, learning_rate 0.00360342
2020-12-08T14:51:24.042918: step 82, loss 0.977152, acc 0.65625, learning_rate 0.00359873
2020-12-08T14:51:24.657919: step 83, loss 0.787089, acc 0.5625, learning_rate 0.00359404
2020-12-08T14:51:25.277957: step 84, loss 1.0561, acc 0.46875, learning_rate 0.00358936
2020-12-08T14:51:25.862045: step 85, loss 1.14644, acc 0.53125, learning_rate 0.00358469
2020-12-08T14:51:26.440676: step 86, loss 0.699028, acc 0.6875, learning_rate 0.00358002
2020-12-08T14:51:27.029534: step 87, loss 0.817064, acc 0.625, learning_rate 0.00357536
2020-12-08T14:51:27.648084: step 88, loss 0.727322, acc 0.59375, learning_rate 0.0035707
2020-12-08T14:51:28.225341: step 89, loss 0.800712, acc 0.6875, learning_rate 0.00356605
2020-12-08T14:51:28.823034: step 90, loss 0.626553, acc 0.6875, learning_rate 0.00356141
2020-12-08T14:51:29.4

2020-12-08T14:52:22.966656: step 167, loss 0.675734, acc 0.6875, learning_rate 0.00322196
2020-12-08T14:52:23.553653: step 168, loss 0.543955, acc 0.71875, learning_rate 0.00321778
2020-12-08T14:52:24.159153: step 169, loss 0.731723, acc 0.6875, learning_rate 0.0032136
2020-12-08T14:52:24.922116: step 170, loss 0.57034, acc 0.78125, learning_rate 0.00320943
2020-12-08T14:52:25.683621: step 171, loss 0.737064, acc 0.5625, learning_rate 0.00320527
2020-12-08T14:52:26.402352: step 172, loss 0.661111, acc 0.6875, learning_rate 0.00320111
2020-12-08T14:52:27.063352: step 173, loss 0.480949, acc 0.78125, learning_rate 0.00319695
2020-12-08T14:52:27.660402: step 174, loss 0.54257, acc 0.71875, learning_rate 0.0031928
2020-12-08T14:52:28.245146: step 175, loss 0.713441, acc 0.6875, learning_rate 0.00318866
2020-12-08T14:52:28.822909: step 176, loss 0.589365, acc 0.78125, learning_rate 0.00318452
2020-12-08T14:52:29.422262: step 177, loss 0.797148, acc 0.71875, learning_rate 0.00318039
2020-12-

2020-12-08T14:53:24.333928: step 254, loss 0.806407, acc 0.65625, learning_rate 0.00287831
2020-12-08T14:53:24.937423: step 255, loss 0.592705, acc 0.6875, learning_rate 0.00287458
2020-12-08T14:53:25.520939: step 256, loss 0.759515, acc 0.75, learning_rate 0.00287087
2020-12-08T14:53:26.092386: step 257, loss 0.73943, acc 0.59375, learning_rate 0.00286716
2020-12-08T14:53:26.672887: step 258, loss 0.740099, acc 0.6875, learning_rate 0.00286345
2020-12-08T14:53:27.248489: step 259, loss 0.648174, acc 0.6875, learning_rate 0.00285975
2020-12-08T14:53:27.838617: step 260, loss 0.64668, acc 0.78125, learning_rate 0.00285605
2020-12-08T14:53:28.453120: step 261, loss 0.800912, acc 0.65625, learning_rate 0.00285236
2020-12-08T14:53:29.037488: step 262, loss 0.530312, acc 0.78125, learning_rate 0.00284867
2020-12-08T14:53:29.610050: step 263, loss 0.883757, acc 0.5625, learning_rate 0.00284499
2020-12-08T14:53:30.203710: step 264, loss 0.654939, acc 0.75, learning_rate 0.00284131
2020-12-08T

2020-12-08T14:54:19.484449: step 340, loss 0.441786, acc 0.78125, learning_rate 0.0025758
2020-12-08T14:54:20.067533: step 341, loss 0.523413, acc 0.75, learning_rate 0.00257248
2020-12-08T14:54:20.656370: step 342, loss 0.468944, acc 0.78125, learning_rate 0.00256917
2020-12-08T14:54:21.241406: step 343, loss 0.413869, acc 0.78125, learning_rate 0.00256586
2020-12-08T14:54:21.806899: step 344, loss 0.369732, acc 0.90625, learning_rate 0.00256256
2020-12-08T14:54:22.376866: step 345, loss 0.524419, acc 0.8125, learning_rate 0.00255926
2020-12-08T14:54:22.968365: step 346, loss 0.70113, acc 0.65625, learning_rate 0.00255596
2020-12-08T14:54:23.541897: step 347, loss 0.477685, acc 0.71875, learning_rate 0.00255268
2020-12-08T14:54:24.112402: step 348, loss 0.315274, acc 0.875, learning_rate 0.00254939
2020-12-08T14:54:24.693905: step 349, loss 0.457055, acc 0.78125, learning_rate 0.00254611
2020-12-08T14:54:25.287135: step 350, loss 0.434873, acc 0.78125, learning_rate 0.00254283

Evalua

...

2020-12-08T15:19:56.140978: step 2581, loss 0.190083, acc 0.84375, learning_rate 0.000222776
2020-12-08T15:19:56.752442: step 2582, loss 0.0788593, acc 0.9375, learning_rate 0.000222611
2020-12-08T15:19:57.387089: step 2583, loss 0.188577, acc 0.875, learning_rate 0.000222447
2020-12-08T15:19:58.070088: step 2584, loss 0.113591, acc 0.96875, learning_rate 0.000222283
2020-12-08T15:19:58.642054: step 2585, loss 0.324162, acc 0.875, learning_rate 0.000222119
2020-12-08T15:19:59.200556: step 2586, loss 0.0849848, acc 1, learning_rate 0.000221956
2020-12-08T15:19:59.787557: step 2587, loss 0.0892662, acc 0.96875, learning_rate 0.000221792
2020-12-08T15:20:00.360554: step 2588, loss 0.170417, acc 0.96875, learning_rate 0.000221629
2020-12-08T15:20:00.899891: step 2589, loss 0.133814, acc 0.9375, learning_rate 0.000221466
2020-12-08T15:20:01.438588: step 2590, loss 0.144588, acc 0.9375, learning_rate 0.000221303
2020-12-08T15:20:02.012626: step 2591, loss 0.185307, acc 0.90625, learning_rate

2020-12-08T15:21:01.858805: step 2666, loss 0.44916, acc 0.78125, learning_rate 0.000209554
2020-12-08T15:21:02.490840: step 2667, loss 0.493154, acc 0.8125, learning_rate 0.000209408
2020-12-08T15:21:03.118569: step 2668, loss 0.192206, acc 0.9375, learning_rate 0.000209261
2020-12-08T15:21:03.690904: step 2669, loss 0.191315, acc 0.9375, learning_rate 0.000209115
2020-12-08T15:21:04.234367: step 2670, loss 0.185815, acc 0.875, learning_rate 0.000208968
2020-12-08T15:21:04.846365: step 2671, loss 0.175708, acc 0.90625, learning_rate 0.000208823
2020-12-08T15:21:05.392829: step 2672, loss 0.172309, acc 0.90625, learning_rate 0.000208677
2020-12-08T15:21:05.972306: step 2673, loss 0.27173, acc 0.84375, learning_rate 0.000208531
2020-12-08T15:21:06.540062: step 2674, loss 0.217672, acc 0.9375, learning_rate 0.000208386
2020-12-08T15:21:07.132422: step 2675, loss 0.0794648, acc 1, learning_rate 0.000208241
2020-12-08T15:21:07.840981: step 2676, loss 0.111204, acc 1, learning_rate 0.000208

2020-12-08T15:22:05.108677: step 2751, loss 0.129908, acc 0.9375, learning_rate 0.000197757
2020-12-08T15:22:05.713428: step 2752, loss 0.150241, acc 0.9375, learning_rate 0.000197626
2020-12-08T15:22:06.285425: step 2753, loss 0.148246, acc 1, learning_rate 0.000197495
2020-12-08T15:22:06.832264: step 2754, loss 0.271007, acc 0.875, learning_rate 0.000197364
2020-12-08T15:22:07.540031: step 2755, loss 0.115103, acc 0.96875, learning_rate 0.000197234
2020-12-08T15:22:08.251940: step 2756, loss 0.132108, acc 0.96875, learning_rate 0.000197104
2020-12-08T15:22:08.953904: step 2757, loss 0.3638, acc 0.8125, learning_rate 0.000196974
2020-12-08T15:22:09.567440: step 2758, loss 0.187547, acc 0.875, learning_rate 0.000196844
2020-12-08T15:22:10.163443: step 2759, loss 0.276954, acc 0.875, learning_rate 0.000196714
2020-12-08T15:22:10.772405: step 2760, loss 0.257508, acc 0.8125, learning_rate 0.000196584
2020-12-08T15:22:11.469442: step 2761, loss 0.365969, acc 0.84375, learning_rate 0.00019

2020-12-08T15:23:10.157507: step 2838, loss 0.247095, acc 0.875, learning_rate 0.000186996
2020-12-08T15:23:10.770583: step 2839, loss 0.191769, acc 0.90625, learning_rate 0.000186879
2020-12-08T15:23:11.668510: step 2840, loss 0.198387, acc 0.8125, learning_rate 0.000186763
2020-12-08T15:23:12.307054: step 2841, loss 0.281255, acc 0.90625, learning_rate 0.000186647
2020-12-08T15:23:12.888534: step 2842, loss 0.150751, acc 0.9375, learning_rate 0.000186531
2020-12-08T15:23:13.523998: step 2843, loss 0.117909, acc 0.9375, learning_rate 0.000186415
2020-12-08T15:23:14.131523: step 2844, loss 0.23137, acc 0.875, learning_rate 0.000186299
2020-12-08T15:23:14.721190: step 2845, loss 0.302242, acc 0.8125, learning_rate 0.000186184
2020-12-08T15:23:15.308653: step 2846, loss 0.279654, acc 0.875, learning_rate 0.000186068
2020-12-08T15:23:16.035661: step 2847, loss 0.317375, acc 0.875, learning_rate 0.000185953
2020-12-08T15:23:16.832619: step 2848, loss 0.280392, acc 0.90625, learning_rate 0.

2020-12-08T15:24:17.265294: step 2923, loss 0.3244, acc 0.875, learning_rate 0.000177628
2020-12-08T15:24:17.931663: step 2924, loss 0.435987, acc 0.8125, learning_rate 0.000177524
2020-12-08T15:24:18.647134: step 2925, loss 0.0745711, acc 0.96875, learning_rate 0.00017742
2020-12-08T15:24:19.242669: step 2926, loss 0.203735, acc 0.875, learning_rate 0.000177316
2020-12-08T15:24:19.794136: step 2927, loss 0.156023, acc 0.90625, learning_rate 0.000177213
2020-12-08T15:24:20.475134: step 2928, loss 0.10318, acc 0.9375, learning_rate 0.000177109
2020-12-08T15:24:21.077136: step 2929, loss 0.250857, acc 0.875, learning_rate 0.000177006
2020-12-08T15:24:21.785135: step 2930, loss 0.148402, acc 0.9375, learning_rate 0.000176903
2020-12-08T15:24:22.471634: step 2931, loss 0.321044, acc 0.875, learning_rate 0.0001768
2020-12-08T15:24:23.097684: step 2932, loss 0.221416, acc 0.96875, learning_rate 0.000176697
2020-12-08T15:24:23.700167: step 2933, loss 0.112963, acc 0.96875, learning_rate 0.000

In [45]:
dataset_name = cfg["datasets"]["default"]
print('dataset_name: ', dataset_name)
if FLAGS.eval_train:
    if dataset_name == "tobacco":
        print("Loading data !")
        # ====================================================================================
        # added:
        data_path = os.path.join(ROOT_DIR, cfg["datasets"][dataset_name]['parent_dir'] + '/')
        print("*************data_path", data_path)
        datasets = data_helpers.get_datasets_tobacco(data_path)
        # ====================================================================================
    elif dataset_name == "localdata":
        datasets = data_helpers.get_datasets_localdata(container_path=cfg["datasets"][dataset_name]["container_path"],
                                                       categories=cfg["datasets"][dataset_name]["categories"],
                                                       shuffle=cfg["datasets"][dataset_name]["shuffle"],
                                                       random_state=cfg["datasets"][dataset_name]["random_state"])
        
    x_raw, y_test = data_helpers.load_data_labels(datasets)
    y_test = np.argmax(y_test, axis=1)
    print("Total number of test examples: {}".format(len(y_test)))
else:
    print("Flow shouldn't be here.")

dataset_name:  tobacco
Loading data !
*************data_path C:\Users\User\Lightweighted-CNN-for-Document-Classification\data/tobacco-data/
['0.txt', '6.txt', '8.txt']
The Data before flattening:  [['SURGEON GENERAL’S WARNING: Smoking', 'By Pregnant Women May Result in Fetal', 'injury, Premature Birth, And Low Birth Weight.', 'A r 7', '<a:u:', 'Jimmy’s Grille', '32!] Snullr Birch 322-5334', 'BARS “HER“ LIVE MUS_IC', "Gel it uin' will] the lunk~ azz sounds n1 « MW ' -", 'the ﬁne and only Dnlsaliu. n’u keep ‘ SNAKE PIT SynergY-ﬂhe “W New “3 b9- CRICKET on 11-:', '. your heoly shakin’ until lhe wee hours. ace E. l th Ave. 8 l-la r. Fridays, lfs B_alllsuc with high no. Easy , m', 'not“ misslhis show on Tuesday, Snake Pit Fridays introduces the energy dance with DIs Amtrax. Sizzlin’ Sepgem', 'Saplemher 30. Brnughiin you by Camel. home grown mlent of Miss Saturdays, Shake! Club, dance, ai‘dweﬁlml', 'Chocolate Thunderpussy. house with Anthrayngribgﬂ . ‘', 'Feast your eyes on chis spectacw Rlc

Total number of test examples: 11936


In [57]:
!python eval_SGD_small.py --eval_train --checkpoint_dir="./runs/1607457019/checkpoints/"

***********************YML_PATH C:\Users\User\Lightweighted-CNN-for-Document-Classification\train/helper/config.yml
dataset_name:  tobacco
Loading data !
*************data_path C:\Users\User\Lightweighted-CNN-for-Document-Classification\data/tobacco-data/
['0.txt', '6.txt', '8.txt']
Total number of test examples: 11936
Vocabulary Size: 16099
Train/Dev split: 9549/2387
size of x_dev, y_dev: 2387 2387

Evaluating...



C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [64]:
graph = tf.Graph()
g=tf.compat.v1.get_default_graph()
with graph.as_default():
    # tf.compat.v1.contrib.quantize.create_eval_graph(input_graph=g)
    tf.contrib.quantize.create_eval_graph(input_graph=g)
    session_conf = tf.compat.v1.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.compat.v1.Session(config=session_conf)
    # sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.compat.v1.train.import_meta_graph("{}.meta".format("./runs/1607457019/checkpoints/model-2950"))
    # ==================================================================================
        saver.restore(sess, "./runs/1607457019/checkpoints/model-2950")

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        scores = graph.get_operation_by_name("output/scores").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = data_helpers.batch_iter(list(x_dev), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []
        all_probabilities = None

        for x_dev_batch in batches:
            batch_predictions_scores = sess.run([predictions, scores], {input_x: x_dev_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions_scores[0]])
            probabilities = softmax(batch_predictions_scores[1])
            if all_probabilities is not None:
                all_probabilities = np.concatenate([all_probabilities, probabilities])
            else:
                all_probabilities = probabilities

# Print accuracy if y_dev is defined
if y_dev is not None:
    correct_predictions = float(sum(all_predictions == y_dev))
    print("Total number of test examples: {}".format(len(y_dev)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_dev))))
    print(metrics.classification_report(y_dev, all_predictions, target_names=datasets['target_names']))
    print(metrics.confusion_matrix(y_dev, all_predictions))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./runs/1607457019/checkpoints/model-2950


FailedPreconditionError: Attempting to use uninitialized value batch_normalization_3/moving_mean
	 [[node batch_normalization_3/moving_mean/read (defined at <ipython-input-64-6ad6a7f35ed7>:13) ]]

Original stack trace for 'batch_normalization_3/moving_mean/read':
  File "C:\Users\User\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\User\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\User\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\User\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "C:\Users\User\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\User\Ana